# Importing libraries

In [83]:
import pandas as pd
import numpy as np

# Importing data

In [84]:
df = pd.read_csv('../data/data.csv', encoding='latin1')
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


## Converting types

Converting the `InvoiceDate` columns to a date type and `CustomerID` to string, since it's an ID.

In [85]:
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
df['CustomerID'] = df['CustomerID'].fillna(0).astype(int)

# Handling missing and duplicated values

We will map the missing descriptions and try to fill them with `StockCode` corresponding description rows. If not found, fill it with *UNKNOWN PRODUCT*.

In [86]:
description_mapping = df.dropna(subset=['Description']).drop_duplicates('StockCode').set_index('StockCode')['Description']

df_mis_dedup = df.copy()
print(f'Before mapping: {df_mis_dedup['Description'].isna().sum()}')
df_mis_dedup['Description'] = df_mis_dedup['Description'].fillna(df_mis_dedup['StockCode'].map(description_mapping))
print(f'After mapping: {df_mis_dedup['Description'].isna().sum()}')
df_mis_dedup['Description'] = df_mis_dedup['Description'].fillna('UNKNOWN PRODUCT')

Before mapping: 1454
After mapping: 112


We will also drop full duplicated rows.
We are considering a duplicated row as a row where every value of all the columns are the same as in another instance.

In [87]:
print(f'Duplicated values: {df_mis_dedup.duplicated().sum()}.')

df_mis_dedup = df_mis_dedup.drop_duplicates()

Duplicated values: 5268.


# Flagging "abnormal" values

Flagging missing `CustomerID` values (which we replaced by '0') to represent *Guest* buyers

In [88]:
df_flag = df_mis_dedup.copy()
df_flag['CustomerType'] = 'Client'
df_flag.loc[df_flag['CustomerID'] == 0, 'CustomerType'] = 'Guest'

Flagging negative `Quantity` and `UnitPrice` rows to represent returned purchases/discarded products and accounting adjusts, respectively.

In [89]:
df_flag['TransactionType'] = 'Sale'
df_flag.loc[df_flag['Quantity'] < 0, 'TransactionType'] = 'Return/Cancellation'
df_flag.loc[df_flag['UnitPrice'] < 0, 'TransactionType'] = 'Adjustment'

# Creating new business columns

In [90]:
df_flag['TotalAmount'] = df_flag['Quantity'] * df_flag['UnitPrice']
df_flag.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,CustomerType,TransactionType,TotalAmount
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,Client,Sale,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,Client,Sale,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,Client,Sale,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,Client,Sale,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,Client,Sale,20.34


# Normalizing 
Let's normalize the wide table into several ones to avoid redundancy

### Countries table

In [91]:
country_map = {
    'United Kingdom': 'UK',
    'France': 'FR',
    'Australia': 'AU',
    'Netherlands': 'NL',
    'Germany': 'DE',
    'Norway': 'NO',
    'EIRE': 'IE',
    'Switzerland': 'CH',
    'Spain': 'ES',
    'Poland': 'PL',
    'Portugal': 'PT',
    'Italy': 'IT',
    'Belgium': 'BE',
    'Lithuania': 'LT',
    'Japan': 'JP',
    'Iceland': 'IS',
    'Channel Islands': 'CI',
    'Denmark': 'DK',
    'Cyprus': 'CY',
    'Sweden': 'SE',
    'Austria': 'AT',
    'Israel': 'IL',
    'Finland': 'FI',
    'Bahrain': 'BH',
    'Greece': 'GR',
    'Hong Kong': 'HK',
    'Singapore': 'SG',
    'Lebanon': 'LB',
    'United Arab Emirates': 'AE',
    'Saudi Arabia': 'SA',
    'Czech Republic': 'CZ',
    'Canada': 'CA',
    'Unspecified': 'UNS',
    'Brazil': 'BR',
    'USA': 'US',
    'European Community': 'EC',
    'Malta': 'MT',
    'RSA': 'ZA'
}

df_countryid = df_flag.copy()
df_countryid['CountryID'] = df_countryid['Country'].map(country_map)

countries_table = df_countryid[['CountryID', 'Country']].groupby('CountryID').first().reset_index()
countries_table = countries_table.rename(columns={'Country': 'CountryName'})
countries_table.head()

,CountryID,CountryName
0,AE,United Arab Emirates
1,AT,Austria
2,AU,Australia
3,BE,Belgium
4,BH,Bahrain


### Invoice table

In [92]:
invoices_table = df_countryid[['InvoiceNo', 'InvoiceDate', 'CustomerID']]
invoices_table = invoices_table.groupby('InvoiceNo').first().reset_index()
invoices_table.head()

,InvoiceNo,InvoiceDate,CustomerID
0,536365,2010-12-01 08:26:00,17850
1,536366,2010-12-01 08:28:00,17850
2,536367,2010-12-01 08:34:00,13047
3,536368,2010-12-01 08:34:00,13047
4,536369,2010-12-01 08:35:00,13047


### Customers table

In [93]:
customers_table = df_countryid[['CustomerID', 'CountryID', 'CustomerType']]
customers_table = customers_table.groupby('CustomerID').first().reset_index()
customers_table.head()

,CustomerID,CountryID,CustomerType
0,0,UK,Guest
1,12346,UK,Client
2,12347,IS,Client
3,12348,FI,Client
4,12349,IT,Client


### Products table

In [94]:
products_table = df_countryid[['StockCode', 'Description']]
products_table = products_table.groupby('StockCode')['Description'].apply(lambda x: x.mode()[0]).reset_index()
products_table.head()

,StockCode,Description
0,10002,INFLATABLE POLITICAL GLOBE
1,10080,GROOVY CACTUS INFLATABLE
2,10120,DOGGY RUBBER
3,10123C,HEARTS WRAPPING TAPE
4,10123G,UNKNOWN PRODUCT


### Transactions table

We will create `LineNo` to represent the line item of the correspondent invoice. This will help us creating the composite key for ingestion into the database .

In [95]:
transactions_table = df_countryid.copy()
transactions_table = transactions_table[['InvoiceNo', 'StockCode', 'Quantity', 'UnitPrice', 'TotalAmount', 'TransactionType']]
transactions_table['LineNo'] = transactions_table.groupby('InvoiceNo').cumcount() + 1
transactions_table.head()

,InvoiceNo,StockCode,Quantity,UnitPrice,TotalAmount,TransactionType,LineNo
0,536365,85123A,6,2.55,15.30,Sale,1
1,536365,71053,6,3.39,20.34,Sale,2
2,536365,84406B,8,2.75,22.00,Sale,3
3,536365,84029G,6,3.39,20.34,Sale,4
4,536365,84029E,6,3.39,20.34,Sale,5
